In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/Triple_Construction

/content/drive/MyDrive/Colab Notebooks/Triple_Construction


In [ ]:
ls

 1_triple_construction.ipynb
 2_triple_count_256seg.ipynb
 2_triple_count.ipynb
'Copy of topic4_30000sents_entityData_finalModel.csv'
 entity_data.csv
 entity_dataset.csv
 kg_df_finalResult_256seg.csv
 kg_df_finalResult.csv
 original_triple_construction_test2.ipynb
 preprocessed_topic4_triples_256seg.csv
 preprocessed_topic4_triples.csv
 topic4_5000sents_triples.csv
 verb_data.csv
 verb_dataset.csv


In [ ]:
import time

start_time = time.time()

In [ ]:
import pandas as pd

In [ ]:
# original entity data file name = taken_ner_result.csv
# original relation data file name = topic4_verb_cluster_data_5000sents.csv

In [ ]:
entity_df = pd.read_csv('entity_dataset.csv')
entity_df.head()

,Unnamed: 0,doc_id,sent_id,word_id,type,word
0,0,0,0,10,DISEASE_OR_SYNDROME,submacular hemorrhage
1,1,0,3,3,DISEASE_OR_SYNDROME,submacular hemorrhage
2,2,0,9,8,DISEASE_OR_SYNDROME,retinal hemorrhage
3,3,2,2,6,DISEASE_OR_SYNDROME,melanoma lung
4,4,2,2,10,DISEASE_OR_SYNDROME,head neck


In [ ]:
entity_df['doc_id'] = entity_df['doc_id'].fillna(0).astype(int)
entity_df['sent_id'] = entity_df['sent_id'].fillna(0).astype(int)
entity_df['word_id'] = entity_df['word_id'].fillna(0).astype(int)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

Lem = WordNetLemmatizer()

In [ ]:
entity_df.drop(entity_df[entity_df.isna().any(axis=1)].index, axis=0, inplace=True)

Lemmatized Entity

In [ ]:
entity_df['word_lemma'] = entity_df['word'].apply(lambda x: Lem.lemmatize(x))

In [ ]:
entity_df.head()

,Unnamed: 0,doc_id,sent_id,word_id,type,word,word_lemma
0,0,0,0,10,DISEASE_OR_SYNDROME,submacular hemorrhage,submacular hemorrhage
1,1,0,3,3,DISEASE_OR_SYNDROME,submacular hemorrhage,submacular hemorrhage
2,2,0,9,8,DISEASE_OR_SYNDROME,retinal hemorrhage,retinal hemorrhage
3,3,2,2,6,DISEASE_OR_SYNDROME,melanoma lung,melanoma lung
4,4,2,2,10,DISEASE_OR_SYNDROME,head neck,head neck


In [ ]:
verb_df = pd.read_csv('verb_dataset.csv')
verb_df.head()

,Unnamed: 0,Unnamed: 0.1,doc_id,sent_id,verb_id,verb,check_verb,main_verb,lemma_verb,cluster_title
0,0,0,0,0,36,was conducted,"[(9148364968459358513, 1, 2)]",conducted,conduct,induce
1,1,0,0,0,11,related,"[(9148364968459358513, 0, 1)]",related,relate,relate
2,2,1,0,1,13,were reviewed,"[(9148364968459358513, 1, 2)]",reviewed,review,review
3,3,1,0,1,2,including,"[(9148364968459358513, 0, 1)]",including,include,include
4,4,2,0,2,7,were included,"[(9148364968459358513, 1, 2)]",included,include,include


In [ ]:
### number of documents = 495
### number of sentences = 5000

In [ ]:
# get all the possible doc_id & sent_id

doc_sent_ids = entity_df.merge(verb_df, on = ['doc_id', 'sent_id'])[['doc_id', 'sent_id']]
doc_sent_ids = doc_sent_ids.drop_duplicates()
doc_sent_ids.reset_index(inplace = True)
doc_sent_ids.head()

,index,doc_id,sent_id
0,0,0,0
1,12,0,3
2,18,2,2
3,28,2,3
4,36,3,0


In [ ]:
triple_df = pd.DataFrame()

In [ ]:
# subset (sent number)

#subset_length = len(doc_sent_ids)
#subset_length = 100

In [ ]:
# triple construction

triple_idx = 0
for idx in range(len(doc_sent_ids)):
  # print(idx)
  doc_id = doc_sent_ids.loc[idx, 'doc_id']
  sent_id = doc_sent_ids.loc[idx, 'sent_id']
  
  # print('doc_id: ', doc_id)
  # print('sent_id: ', sent_id)

  sub_entity_df = entity_df[(entity_df['doc_id'] == doc_id) & (entity_df['sent_id'] == sent_id)]
  sub_verb_df = verb_df[(verb_df['doc_id'] == doc_id) & (verb_df['sent_id'] == sent_id)]

  entity_idx_list = sub_entity_df['word_id'].tolist()
  verb_idx_list = sub_verb_df['verb_id'].tolist()

  triple = []
  for vp_idx in verb_idx_list:
      head_list = [np_idx for np_idx in entity_idx_list if(np_idx < vp_idx)]
      tail_list = [np_idx for np_idx in entity_idx_list if(np_idx > vp_idx)]

      if (not head_list) or (not tail_list):
          continue

      for head_idx in head_list:
          for tail_idx in tail_list:

            triple_df.loc[triple_idx, 'doc_id'] = doc_id
            triple_df.loc[triple_idx, 'sent_id'] = sent_id

            head_str = sub_entity_df[sub_entity_df['word_id'] == head_idx]['word'].iloc[0]
            head_lemma = sub_entity_df[sub_entity_df['word_id'] == head_idx]['word_lemma'].iloc[0]
            head_type = sub_entity_df[sub_entity_df['word_id'] == head_idx]['type'].iloc[0]

            tail_str = sub_entity_df[sub_entity_df['word_id'] == tail_idx]['word'].iloc[0]
            tail_lemma = sub_entity_df[sub_entity_df['word_id'] == tail_idx]['word_lemma'].iloc[0]
            tail_type = sub_entity_df[sub_entity_df['word_id'] == tail_idx]['type'].iloc[0]

            verb_str = sub_verb_df[sub_verb_df['verb_id'] == vp_idx]['verb'].iloc[0]
            lemma_verb = sub_verb_df[sub_verb_df['verb_id'] == vp_idx]['lemma_verb'].iloc[0]
            cluster_verb = sub_verb_df[sub_verb_df['verb_id'] == vp_idx]['cluster_title'].iloc[0]

            if head_lemma == tail_lemma:   
              continue
            
            else:

              triple_df.loc[triple_idx, 'triple'] = str(head_lemma) + ' ' + str(cluster_verb) + ' ' + str(tail_lemma)

              triple_df.loc[triple_idx, 'head_idx'] = head_idx
              triple_df.loc[triple_idx, 'verb_idx'] = vp_idx
              triple_df.loc[triple_idx, 'tail_idx'] = tail_idx
              
              triple_df.loc[triple_idx, 'head_str'] = head_str
              triple_df.loc[triple_idx, 'head_lemma'] = head_lemma
              triple_df.loc[triple_idx, 'head_type'] = head_type

              triple_df.loc[triple_idx, 'tail_str'] = tail_str
              triple_df.loc[triple_idx, 'tail_lemma'] = tail_lemma
              triple_df.loc[triple_idx, 'tail_type'] = tail_type
              
              triple_df.loc[triple_idx, 'main_verb'] = verb_str
              triple_df.loc[triple_idx, 'lemma_verb'] = lemma_verb
              triple_df.loc[triple_idx, 'cluster_verb'] = cluster_verb

              triple_idx = triple_idx + 1

# break

In [ ]:
triple_df.drop(triple_df[triple_df.isna().any(axis=1)].index, axis=0, inplace=True)

In [ ]:
triple_df.head()

,doc_id,sent_id,triple,head_idx,verb_idx,tail_idx,head_str,head_lemma,head_type,tail_str,tail_lemma,tail_type,main_verb,lemma_verb,cluster_verb
0,0.0,0.0,submacular hemorrhage relate sars-cov-2,10.0,11.0,14.0,submacular hemorrhage,submacular hemorrhage,DISEASE_OR_SYNDROME,sars-cov-2,sars-cov-2,CORONAVIRUS,related,relate,relate
1,0.0,0.0,submacular hemorrhage relate vaccine,10.0,11.0,15.0,submacular hemorrhage,submacular hemorrhage,DISEASE_OR_SYNDROME,vaccines,vaccine,CHEMICAL,related,relate,relate
2,0.0,0.0,age-related relate sars-cov-2,5.0,11.0,14.0,age-related,age-related,GENE_OR_GENOME,sars-cov-2,sars-cov-2,CORONAVIRUS,related,relate,relate
3,0.0,0.0,age-related relate vaccine,5.0,11.0,15.0,age-related,age-related,GENE_OR_GENOME,vaccines,vaccine,CHEMICAL,related,relate,relate
4,0.0,0.0,macular relate sars-cov-2,6.0,11.0,14.0,macular,macular,TISSUE,sars-cov-2,sars-cov-2,CORONAVIRUS,related,relate,relate


In [ ]:
triple_df.to_csv('preprocessed_topic4_triples.csv')

In [ ]:
end_time = time.time()

print('Estimate runtime: ', (end_time-start_time)/60, ' minutes.')

Estimate runtime:  6.680422226587932  minutes.


In [ ]:
### for all

# Estimate runtime:  6.680422226587932  minutes.